In [ ]:
%run -i "common.py"

In [ ]:
%%capture
### Load File Universal:
data_folder = Path("/Users/pyuan/EEG-artifact-rejection/601/")
file_name = data_folder / "Rew_601_rest_bb_epoch.set"
epochs = mne.io.read_epochs_eeglab(file_name)
data = epochs.to_data_frame()

In [ ]:
%%capture
### Quick Clean DataFrame:
df = data.copy(); df = df.reset_index(); df = df.drop(['condition'], axis=1);

In [ ]:
%%capture
### IForest Algorithm:
df_ = df.copy(); df_ = df_.drop(['epoch', 'time'], axis=1)
X = df_.values
clfIF = IsolationForest(random_state=42, contamination=0.00030, n_jobs=-1, max_samples=1.0)
clfIF.fit(X)
pred_train, pred_test = clfIF.predict(X), clfIF.predict(X)
count_train, count_test = np.unique(ar=pred_train, return_counts=True), np.unique(ar=pred_test, return_counts=True)
index_train, index_test = [i for i,x in enumerate(pred_train) if x == -1] , [i for i,x in enumerate(pred_test) if x == -1]
df_IF = df_.loc[index_test]
num_anomalies = count_train[1][0], count_test[1][0]; total_pts = count_train[1][1], count_test[1][1]

In [ ]:
%%capture
### True Rejection Percentage:
reject_arr = sio.loadmat('/Users/pyuan/EEG-artifact-rejection/601//Rew_601_rest_reject_rmm.mat')
reject_key = reject_arr['reject'][0].tolist()